## RAG Evaluation

In [8]:
%load_ext autoreload
%autoreload 2
import nest_asyncio

nest_asyncio.apply()
import sys
import os
from pathlib import Path
import time
import json
from typing import Dict, Any

# Add the src directory to Python path
sys.path.append('../src/')

#from simple_rag.parsers.unstructured_parser import UnstructuredParserProcessor


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from simple_rag.evaluation.ragas import RAGEvaluator

import time

evaluator = RAGEvaluator(
        collection_name="simple_rag",
        embedding_model="BAAI/bge-base-en-v1.5",
        llm_model="qwen2.5:7b-instruct",              # Your RAG's LLM
        judge_llm_model="gemini-1.5-flash", # The LLM that *scores*
        judge_llm_provider="gemini",        # Use Gemini
        retriever_k=5                       # Explicitly set K
    )


JSON_PATH = "../src/simple_rag/evaluation/pair_answers.json"
try:

    start_time = time.time()
    evaluator.warmup_models()
    end_time = time.time()
    print(f"Time to load the llm models: {end_time - start_time} seconds")
    evaluator.generate_rag_outputs(JSON_PATH)


    evaluator.save_rag_outputs("../src/simple_rag/evaluation/pair_answers_rag_rerank.json")


except Exception as e:
    print(f"Error generating RAG outputs: {str(e)}")
    

Using Gemini API for RAGAS judge: gemini-1.5-flash
Stopping Ollama server...
✓ Ollama server stopped

[Warmup] Warming up RAG system LLM...

[Phase 1] Initializing RAG system...
  - Collection: simple_rag
  - Embedding Model: BAAI/bge-base-en-v1.5
  - LLM Model: qwen2.5:7b-instruct
  - Retriever K: 5
✓ Qdrant server is already running
Loading reranker model 'mixedbread-ai/mxbai-rerank-base-v1'...
This may take a moment on first run...
Reranker loaded successfully.
Ollama server not detected, attempting to start...
Starting Ollama server...
✓ Ollama server started successfully (took 2s)
LLM loaded successfully
  ✓ RAG system initialized successfully
Warming up Ollama model 'qwen2.5:7b-instruct'...
✓ Model warmed up. Response: Ready...
  ✓ RAG LLM response: Ready...
  ✓ Warmup complete!

Time to load the llm models: 29.36936378479004 seconds

[Phase 1] Generating RAG outputs...
  - Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
  - Benchmark JSON pa

Now lets try and use a more powerfull model to detect what is the bottleneck of our rag pipeline

In [2]:
from simple_rag.evaluation.ragas import RAGEvaluator

import time

evaluator = RAGEvaluator(
        collection_name="simple_rag",
        embedding_model="BAAI/bge-base-en-v1.5",
        llm_model="qwen2.5:7b-instruct",              # Your RAG's LLM
        judge_llm_model="gemini-1.5-flash", # The LLM that *scores*
        judge_llm_provider="gemini",        # Use Gemini
        retriever_k=5                       # Explicitly set K
    )

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
JSON_PATH = "../src/simple_rag/evaluation/pair_answers.json"
try:

    
    evaluator.generate_rag_outputs_gemini(JSON_PATH, GOOGLE_API_KEY, model_name="gemini-2.5-flash", temperature=0.0)


    evaluator.save_rag_outputs("../src/simple_rag/evaluation/pair_answers_rag_rerank_gemini.json")


except Exception as e:
    print(f"Error generating RAG outputs: {str(e)}")
    

Using Gemini API for RAGAS judge: gemini-1.5-flash

[Phase 1] Generating RAG outputs with Gemini API...
  - Model: gemini-2.5-flash
  - Rate limit: 10 requests/minute (6s delay between calls)
  - Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
  - Benchmark JSON path (provided): ../src/simple_rag/evaluation/pair_answers.json
  ✓ Found benchmark file: /home/alvar/CascadeProjects/windsurf-project/RAG/src/simple_rag/evaluation/pair_answers.json

[Phase 1] Initializing RAG system...
  - Collection: simple_rag
  - Embedding Model: BAAI/bge-base-en-v1.5
  - LLM Model: qwen2.5:7b-instruct
  - Retriever K: 5
✓ Qdrant server is already running
Loading reranker model 'mixedbread-ai/mxbai-rerank-base-v1'...
This may take a moment on first run...
Reranker loaded successfully.
Ollama server not detected, attempting to start...
Starting Ollama server...
✓ Ollama server started successfully (took 1s)
LLM loaded successfully
  ✓ RAG system initialized successfully

## Evaluation with the RAGAS framework

In [ ]:
# Evaluate only first 10 questions for quick testing
from simple_rag.evaluation.ragas import RAGEvaluator

evaluator = RAGEvaluator(
        collection_name="simple_rag",
        embedding_model="BAAI/bge-base-en-v1.5",
        llm_model="llama3.1:8b",
        judge_llm_model="llama3.1:8b",
        judge_llm_provider="ollama",
        ragas_embedding_provider="ollama",
        ollama_embedding_model="nomic-embed-text",
        cache_folder="./cache",
        retriever_k=15,
        rerank_k=3,                      # Explicitly set K
    )

result = evaluator.evaluate_with_ragas(
    rag_output_json_path="../src/simple_rag/evaluation/pair_answers_rag_rerank_gemini.json"
)

Using Ollama for RAGAS judge: llama3.1:8b
Using Ollama embeddings for RAGAS: nomic-embed-text


TypeError: RAGEvaluator.evaluate_with_ragas() got an unexpected keyword argument 'model_name'